# **Profitable App Profiles for the App Store and Google Play Markets**

Our goal for this project is to analyze data to help our developers understand **what type of apps are likely to attract more users and are profitable for the App Store and Google Play markets**. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

## **1. Data Exploration**

In [1]:
# ### Opening the dataset utilizing csv reader library to read csv file
# from csv import reader

# ### The Google Play data set ###
# opened_file = open('googleplaystore.csv')
# read_file = reader(opened_file)
# android = list(read_file)
# android_header = android[0]
# android = android[1:]

# ### The App Store data set ###
# opened_file = open('AppleStore.csv')
# read_file = reader(opened_file)
# ios = list(read_file)
# ios_header = ios[0]
# ios = ios[1:]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

# loading and reading the A1 dataset
data1 = "/content/drive/MyDrive/Colab Notebooks/Practice/googleplaystore.csv"
android = pd.read_csv(data1)

data2 = "/content/drive/MyDrive/Colab Notebooks/Practice/AppleStore.csv"
ios = pd.read_csv(data2)

In [4]:
android.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [5]:
print("Shape of android dataset:", android.shape)
print('Number of rows:', android.shape[0])
print('Number of columns:', android.shape[1])

Shape of android dataset: (10841, 13)
Number of rows: 10841
Number of columns: 13


In [6]:
ios.head()

,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1
1,389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1
2,529479190,Clash of Clans,116476928,USD,0.0,2130805,579,4.5,4.5,9.24.12,9+,Games,38,5,18,1
3,420009108,Temple Run,65921024,USD,0.0,1724546,3842,4.5,4.0,1.6.2,9+,Games,40,5,1,1
4,284035177,Pandora - Music & Radio,130242560,USD,0.0,1126879,3594,4.0,4.5,8.4.1,12+,Music,37,4,1,1


In [7]:
print("Shape of ios dataset:", ios.shape)
print('Number of rows:', ios.shape[0])
print('Number of columns:', ios.shape[1])

Shape of ios dataset: (7197, 16)
Number of rows: 7197
Number of columns: 16


## **2. Data Clean up**

### 2.1. Finding and deleting incorrect data

In [8]:
# Check for missing/null values in each column
missing_values = android.isnull().sum()

# Display columns with missing values
print(missing_values[missing_values > 0])


Rating            1474
Type                 1
Content Rating       1
Current Ver          8
Android Ver          3
dtype: int64


In [9]:
# Check for non-numeric values in the Rating column
invalid_ratings = android[~android['Rating'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notnull()]

# Display rows with invalid ratings
print(invalid_ratings)

                                       App             Category  Rating  \
23                  Mcqueen Coloring pages       ART_AND_DESIGN     NaN   
113              Wrinkles and rejuvenation               BEAUTY     NaN   
123                 Manicure - nail design               BEAUTY     NaN   
126           Skin Care and Natural Beauty               BEAUTY     NaN   
129    Secrets of beauty, youth and health               BEAUTY     NaN   
...                                    ...                  ...     ...   
10824                            Cardio-FR              MEDICAL     NaN   
10825                   Naruto & Boruto FR               SOCIAL     NaN   
10831             payermonstationnement.fr  MAPS_AND_NAVIGATION     NaN   
10835                             FR Forms             BUSINESS     NaN   
10838               Parkinson Exercices FR              MEDICAL     NaN   

      Reviews  Size  Installs  Type Price Content Rating  \
23         61  7.0M  100,000+  Free    

While some apps does not have a rating, its still important to keep these app for the analysis, which means we need to focus on another approach to detecting incorrect data.

The code below detects if there are any values in the Ratings column that are outside the standard rating value (0 - 5), as well as if there are any apps that has lesser than 0 installs and price just for confirmation.

In [10]:
# Ensure 'Rating', 'Installs', and 'Price' are numeric
android['Rating'] = pd.to_numeric(android['Rating'], errors='coerce')
android['Installs'] = pd.to_numeric(android['Installs'].str.replace('[+,]', '', regex=True), errors='coerce')  # Remove commas and '+' signs in 'Installs'
android['Price'] = pd.to_numeric(android['Price'].str.replace('$', '', regex=True), errors='coerce')  # Remove '$' in 'Price'

# Check for ratings out of the 0-5 range
out_of_range_ratings = android[(android['Rating'] < 0) | (android['Rating'] > 5)]

# Check for negative or impossible values in the 'Installs' or 'Price' column
invalid_installs = android[android['Installs'] < 0]
invalid_price = android[android['Price'] < 0]

# Display results
print("Out of range ratings:")
print(out_of_range_ratings)

print("Invalid installs:")
print(invalid_installs)

print("Invalid prices:")
print(invalid_price)

Out of range ratings:
                                           App Category  Rating Reviews  \
10472  Life Made WI-Fi Touchscreen Photo Frame      1.9    19.0    3.0M   

         Size  Installs Type  Price Content Rating             Genres  \
10472  1,000+       NaN    0    NaN            NaN  February 11, 2018   

      Last Updated Current Ver Android Ver  
10472       1.0.19  4.0 and up         NaN  
Invalid installs:
Empty DataFrame
Columns: [App, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, Android Ver]
Index: []
Invalid prices:
Empty DataFrame
Columns: [App, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, Android Ver]
Index: []


Data row 10472 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5, So We must remove this particular data.

In [11]:
# Display the dataset amount with out-of-range ratings
print(len(android))

# Remove rows where the 'Rating' is out of the 0-5 range
android = android[~((android['Rating'] < 0) | (android['Rating'] > 5))]

# Display the cleaned dataset without out-of-range ratings
print(len(android))

10841
10840


### 2.2. Removing duplicate entries

In [12]:
# Check for duplicate rows based on the 'app' column only
duplicate_apps = android[android.duplicated(subset=['App'])]

# Display the duplicate rows
print('Number of duplicate android apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate android apps:', duplicate_apps[:15])

Number of duplicate android apps: 1181


Examples of duplicate android apps:                                              App  Category  Rating Reviews  \
229                 Quick PDF Scanner + OCR FREE  BUSINESS     4.2   80805   
236                                          Box  BUSINESS     4.2  159872   
239                           Google My Business  BUSINESS     4.4   70991   
256                          ZOOM Cloud Meetings  BUSINESS     4.4   31614   
261                    join.me - Simple Meetings  BUSINESS     4.0    6989   
265                                          Box  BUSINESS     4.2  159872   
266                                     Zenefits  BUSINESS     4.2     296   
267                                   Google Ads  BUSINESS     4.3   29313   
268                           Google My Business  BUSINESS     4.4   70991   
269                                        Slack  BUSINESS     4.4   51507   
270                           FreshBooks Classic  BUSINESS     4.

In [13]:
print('expected length:', len(android) - 1181)

expected length: 9659


In [14]:
# Ensure 'Reviews' column is numeric, just in case there are issues with data types
android['Reviews'] = pd.to_numeric(android['Reviews'], errors='coerce')

# Sort the dataset by 'App' and 'Reviews' in descending order (so highest reviews come first)
android_sorted = android.sort_values(by=['App', 'Reviews'], ascending=[True, False])

# Drop duplicates based on 'App', keeping the row with the highest number of reviews
android_clean = android_sorted.drop_duplicates(subset=['App'], keep='first')

# Check for duplicate rows based on the 'app' column only
duplicate_check = android_clean[android_clean.duplicated(subset=['App'])]

print('Number of duplicate android apps:', len(duplicate_check))

Number of duplicate android apps: 0


In [15]:
print("Shape of android_clean dataset:", android_clean.shape)
print('Number of rows:', android_clean.shape[0])
print('Number of columns:', android_clean.shape[1])

Shape of android_clean dataset: (9659, 13)
Number of rows: 9659
Number of columns: 13


the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show the data was collected at different times.

We can use this information to build a criterion for removing the duplicates. The higher the number of reviews, the more recent the data should be. Rather than removing duplicates randomly, we'll only keep the row with the highest number of reviews and remove the other entries for any given app.

### 2.3. Removing non english entries

To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters:

In [16]:
# Function to check if a string has more than 3 non-ASCII characters
def is_english(string):
    non_ascii_count = sum(1 for char in string if ord(char) > 127)
    return non_ascii_count <= 3

# Apply the function to filter out app names with more than 3 non-ASCII characters
android_english = android_clean[android_clean['App'].apply(is_english)]

# Display the cleaned dataset with only mostly English app names
print(android_english)

                                                   App       Category  Rating  \
8884              "i DT" Fútbol. Todos Somos Técnicos.         SPORTS     NaN   
8532                     +Download 4 Instagram Twitter         SOCIAL     4.5   
324                         - Free Comics - Comic Apps         COMICS     3.5   
4541                                                .R          TOOLS     4.5   
4636                                            /u/app  COMMUNICATION     4.7   
...                                                ...            ...     ...   
313                漫咖 Comics - Manga,Novel and Stories         COMICS     4.1   
4362                                        💎 I'm rich      LIFESTYLE     3.8   
2575     💘 WhatsLov: Smileys of love, stickers and GIF         SOCIAL     4.6   
7559  📏 Smart Ruler ↔️ cm/inch measuring for homework!          TOOLS     4.0   
882   🔥 Football Wallpapers 4K | Full HD Backgrounds 😍  ENTERTAINMENT     4.7   

      Reviews  Size   Insta

In [17]:
print("Shape of android_english dataset:", android_english.shape)
print('Number of rows:', android_english.shape[0])
print('Number of columns:', android_english.shape[1])

Shape of android_english dataset: (9614, 13)
Number of rows: 9614
Number of columns: 13


In [18]:
# Apply the same function to filter out non-English track names
ios_english = ios[ios['track_name'].apply(is_english)]

# Display the cleaned dataset with only English track names
print(ios_english)

              id                                    track_name  size_bytes  \
0      284882215                                      Facebook   389879808   
1      389801252                                     Instagram   113954816   
2      529479190                                Clash of Clans   116476928   
3      420009108                                    Temple Run    65921024   
4      284035177                       Pandora - Music & Radio   130242560   
...          ...                                           ...         ...   
7182  1070854722                                  Be-be-bears!   480781312   
7186  1169971902                   Hey Duggee: We Love Animals   136347648   
7192  1170406182  Shark Boom - Challenge Friends with your Pet   245415936   
7194  1070052833                                    Go!Go!Cat!    91468800   
7195  1081295232   Suppin Detective: Expose their true visage!    83026944   

     currency  price  rating_count_tot  rating_count_ver  user_

In [19]:
print("Shape of ios_english dataset:", ios_english.shape)
print('Number of rows:', ios_english.shape[0])
print('Number of columns:', ios_english.shape[1])

Shape of ios_english dataset: (6183, 16)
Number of rows: 6183
Number of columns: 16


## **3. Isolating the Free Apps**

In [20]:
# Ensure the 'Price' column is numeric (removing symbols like '$' if necessary)
if android_english['Price'].dtype == 'object':
    android_english['Price'] = pd.to_numeric(android_english['Price'].str.replace('$', '', regex=True), errors='coerce')

# Filter rows where the 'Price' is 0 (free apps)
android_final = android_english[android_english['Price'] == 0.0]

# Display the final Android dataset with free apps
print(android_final)
print("Shape of android_final dataset:", android_final.shape)
print('Number of rows:', android_final.shape[0])
print('Number of columns:', android_final.shape[1])

                                                    App             Category  \
8884               "i DT" Fútbol. Todos Somos Técnicos.               SPORTS   
8532                      +Download 4 Instagram Twitter               SOCIAL   
324                          - Free Comics - Comic Apps               COMICS   
4541                                                 .R                TOOLS   
4636                                             /u/app        COMMUNICATION   
...                                                 ...                  ...   
3824  乗換NAVITIME　Timetable & Route Search in Japan T...  MAPS_AND_NAVIGATION   
313                 漫咖 Comics - Manga,Novel and Stories               COMICS   
2575      💘 WhatsLov: Smileys of love, stickers and GIF               SOCIAL   
7559   📏 Smart Ruler ↔️ cm/inch measuring for homework!                TOOLS   
882    🔥 Football Wallpapers 4K | Full HD Backgrounds 😍        ENTERTAINMENT   

      Rating  Reviews                Si

In [21]:
# Ensure the 'Price' column is numeric (removing symbols like '$' if necessary)
if ios_english['price'].dtype == 'object':
    ios_english['price'] = pd.to_numeric(ios_english['price'].str.replace('$', '', regex=True), errors='coerce')

# Filter rows where the 'Price' is 0 (free apps)
ios_final = ios_english[ios_english['price'] == 0.0]

# Display the final iOS dataset with free apps
print(ios_final)
print("Shape of ios_final dataset:", ios_final.shape)
print('Number of rows:', ios_final.shape[0])
print('Number of columns:', ios_final.shape[1])

              id                                    track_name  size_bytes  \
0      284882215                                      Facebook   389879808   
1      389801252                                     Instagram   113954816   
2      529479190                                Clash of Clans   116476928   
3      420009108                                    Temple Run    65921024   
4      284035177                       Pandora - Music & Radio   130242560   
...          ...                                           ...         ...   
7164  1065550288          Tchapper Messenger - Keep it private    82187264   
7177  1168960919                            Golden Lotus Slots   111412224   
7192  1170406182  Shark Boom - Challenge Friends with your Pet   245415936   
7194  1070052833                                    Go!Go!Cat!    91468800   
7195  1081295232   Suppin Detective: Expose their true visage!    83026944   

     currency  price  rating_count_tot  rating_count_ver  user_

## Most Common Apps by Genre

our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.


function to generate frequency tables that show percentages

In [22]:
# Function to calculate the frequency table for a specific column in the DataFrame
def freq_table(df, column):
    # Calculate the frequency percentages for each unique value in the column
    value_counts = df[column].value_counts(normalize=True) * 100
    return value_counts

# Function to display the frequency table, sorted in descending order
def display_table(df, column):
    # Get the frequency table
    table = freq_table(df, column)

    # Sort by the percentage values in descending order and display the results
    table_sorted = table.sort_values(ascending=False)
    for index, value in table_sorted.items():
        print(index, ':', value)

We start by examining the frequency table for the prime_genre column of the App Store data set.

In [23]:
display_table(ios_final, 'prime_genre')

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [24]:
# # An alternative to the above code but without reusable function

# # Calculate percentage values in the prime_genre column
# genre_counts = ios_final['prime_genre'].value_counts(normalize=True) * 100

# # Display the percentages with genre names
# print(genre_counts)

We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

display the percentages in a descending order

In [25]:
display_table(android_final, 'Category')

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

In [26]:
display_table(android_final, 'Genres')

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Lifestyle : 3.892148014440433
Productivity : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on the App Store

In [27]:
# Group by 'prime_genre' and calculate the average number of ratings
avg_ratings_df = ios_final.groupby('prime_genre')['rating_count_tot'].mean().reset_index()

# Rename columns for clarity
avg_ratings_df.columns = ['Genre', 'Average Ratings']

# Sort by 'Average Ratings' in descending order
avg_ratings_df = avg_ratings_df.sort_values(by='Average Ratings', ascending=False)

# Display the sorted results
print(avg_ratings_df)

                Genre  Average Ratings
12         Navigation     86090.333333
16          Reference     74942.111111
18  Social Networking     71548.349057
11              Music     57326.530303
22            Weather     52279.892857
0                Book     39758.500000
6        Food & Drink     33333.923077
5             Finance     31467.944444
14      Photo & Video     28441.543750
20             Travel     28243.800000
17           Shopping     26919.690476
8    Health & Fitness     23298.015385
19             Sports     23008.898551
7               Games     22788.669691
13               News     21248.023256
15       Productivity     21028.410714
21          Utilities     18684.456790
9           Lifestyle     16485.764706
4       Entertainment     14029.830709
1            Business      7491.117647
3           Education      7003.983051
2            Catalogs      4004.000000
10            Medical       612.000000


In [28]:
# Filter the DataFrame to get only rows where the 'prime_genre' is 'Navigation'
navigation_apps = ios_final[ios_final['prime_genre'] == 'Navigation']

# Iterate over the filtered DataFrame and print app name and number of ratings
for index, app in navigation_apps.iterrows():
    print(app['track_name'], ':', app['rating_count_tot'])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which has high reviews amount

The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

In [29]:
# Filter the DataFrame where 'prime_genre' is 'Reference'
navigation_apps = ios_final[ios_final['prime_genre'] == 'Reference']

# Print the 'track_name' and 'rating_count_tot' columns for the filtered apps
for index, app in navigation_apps.iterrows():
    print(app['track_name'], ':', app['rating_count_tot'])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

- Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

- Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

- Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

## Most Popular Apps by Genre on the Google Play Store

In [30]:
display_table(android_final, 'Installs') # the Installs columns

1000000.0 : 15.726534296028879
100000.0 : 11.552346570397113
10000000.0 : 10.548285198555957
10000.0 : 10.198555956678701
1000.0 : 8.393501805054152
100.0 : 6.915613718411552
5000000.0 : 6.825361010830325
500000.0 : 5.561823104693141
50000.0 : 4.7721119133574
5000.0 : 4.512635379061372
10.0 : 3.5424187725631766
500.0 : 3.2490974729241873
50000000.0 : 2.3014440433213
100000000.0 : 2.1322202166064983
50.0 : 1.917870036101083
5.0 : 0.78971119133574
1.0 : 0.5076714801444043
500000000.0 : 0.2707581227436823
1000000000.0 : 0.22563176895306858
0.0 : 0.056407942238267145


One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [31]:
# Create a dictionary to hold the average installs for each category
avg_installs_dict = {}

categories_android = freq_table(android_final, 'Category').index

for category in categories_android:
    # Filter to only include apps from the current category
    category_apps = android_final[android_final['Category'] == category]

    # Make Installs column as strings for cleaning
    category_apps.loc[:, 'Installs'] = category_apps['Installs'].astype(str)

    # Removing commas and plus signs, and convert to float for the Installs column
    category_apps.loc[:, 'Installs'] = category_apps['Installs'].str.replace(',', '').str.replace('+', '').astype(float)

    # Calculate total installs and the number of apps in this category
    total_installs = category_apps['Installs'].sum()
    len_category = category_apps.shape[0]

    # Calculate the average number of installs
    avg_n_installs = total_installs / len_category if len_category > 0 else 0

    # Store the average number of installs in the dictionary
    avg_installs_dict[category] = avg_n_installs

# Sort the dictionary by average installs in descending order
sorted_avg_installs = sorted(avg_installs_dict.items(), key=lambda x: x[1], reverse=True)

# Print the sorted categories and their average number of installs
for category, avg in sorted_avg_installs:
    print(f"{category}: {avg}")

COMMUNICATION: 38456119.167247385
VIDEO_PLAYERS: 24727872.452830188
SOCIAL: 23253652.127118643
PHOTOGRAPHY: 17840110.40229885
PRODUCTIVITY: 16787331.344927534
GAME: 15588015.603248259
TRAVEL_AND_LOCAL: 13984077.710144928
ENTERTAINMENT: 11640705.88235294
TOOLS: 10801391.298666667
NEWS_AND_MAGAZINES: 9549178.467741935
BOOKS_AND_REFERENCE: 8767811.894736841
SHOPPING: 7036877.311557789
PERSONALIZATION: 5201482.6122448975
WEATHER: 5074486.197183099
HEALTH_AND_FITNESS: 4188821.9853479853
MAPS_AND_NAVIGATION: 4056941.7741935486
FAMILY: 3695641.8198090694
SPORTS: 3638640.1428571427
ART_AND_DESIGN: 1986335.0877192982
FOOD_AND_DRINK: 1924897.7363636363
EDUCATION: 1833495.145631068
BUSINESS: 1712290.1474201474
LIFESTYLE: 1437816.2687861272
FINANCE: 1387692.475609756
HOUSE_AND_HOME: 1331540.5616438356
DATING: 854028.8303030303
COMICS: 817657.2727272727
AUTO_AND_VEHICLES: 647317.8170731707
LIBRARIES_AND_DEMO: 638503.734939759
PARENTING: 542603.6206896552
BEAUTY: 513151.88679245283
EVENTS: 253542.22

Communication app category

In [32]:
# Filter for apps in the 'COMMUNICATION' category with specific install numbers
filtered_apps = android_final[
    (android_final['Category'] == 'COMMUNICATION') &
    (android_final['Installs'].isin([1_000_000_000, 500_000_000, 100_000_000]))
]

# Print the app names and their install numbers
for index, row in filtered_apps.iterrows():
    print(f"{row['App']}: {row['Installs']:.0f}")  # Format to display as whole numbers

Android Messages: 100000000
BBM - Free Calls & Messages: 100000000
Firefox Browser fast & private: 100000000
GO SMS Pro - Messenger, Free Themes, Emoji: 100000000
Gmail: 1000000000
Google Chrome: Fast & Secure: 1000000000
Google Duo - High Quality Video Calls: 500000000
Hangouts: 1000000000
KakaoTalk: Free Calls & Text: 100000000
Kik: 100000000
LINE: Free Calls & Messages: 500000000
Messenger Lite: Free Calls & Messages: 100000000
Messenger – Text and Video Chat for Free: 1000000000
Opera Browser: Fast and Secure: 100000000
Opera Mini - fast web browser: 100000000
Skype - free IM & video calls: 1000000000
Telegram: 100000000
Truecaller: Caller ID, SMS spam blocking & Dialer: 100000000
UC Browser - Fast Download Private & Secure: 500000000
UC Browser Mini -Tiny Fast Private & Secure: 100000000
Viber Messenger: 500000000
WeChat: 100000000
WhatsApp Messenger: 1000000000
Who: 100000000
Yahoo Mail – Stay Organized: 100000000
imo beta free calls and text: 100000000
imo free video calls and c

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs.

In [33]:
# Filter for communication apps with fewer than 100 million installs
under_100_m = android_final[(android_final['Category'] == 'COMMUNICATION') & (android_final['Installs'] < 100000000)]

# Calculate the average installs
average_installs = under_100_m['Installs'].mean()

print(average_installs)

3603485.3884615386


If these apps were to be removed and the communication apps only consist of apps that has under 100,000,000 installs, the average installs of the communication app category changed from 38,456,119 to 3,603,485.

Video player app category

In [34]:
# Filter for apps in the 'VIDEO_PLAYERS' category with specific install numbers
filtered_apps = android_final[
    (android_final['Category'] == 'VIDEO_PLAYERS') &
    (android_final['Installs'].isin([1_000_000_000, 500_000_000, 100_000_000]))
]

# Print the app names and their install numbers
for index, row in filtered_apps.iterrows():
    print(f"{row['App']}: {row['Installs']:.0f}")  # Format to display as whole numbers

Dubsmash: 100000000
Google Play Movies & TV: 1000000000
MX Player: 500000000
Motorola FM Radio: 100000000
Motorola Gallery: 100000000
VLC for Android: 100000000
VideoShow-Video Editor, Video Maker, Beauty Camera: 100000000
VivaVideo - Video Editor & Photo Movie: 100000000
YouTube: 1000000000


In [35]:
# Filter for VIDEO_PLAYERS apps with fewer than 100 million installs
under_100_m = android_final[(android_final['Category'] == 'VIDEO_PLAYERS') & (android_final['Installs'] < 100000000)]

# Calculate the average installs
average_installs = under_100_m['Installs'].mean()

print(average_installs)

5544878.133333334


We see the same pattern for the video players category, where the market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. the average installs of the communication app category changed from 24,727,872 to 5,544,878.

The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [36]:
# Filter for apps in the 'BOOKS_AND_REFERENCE' category with specific install numbers
filtered_apps = android_final[
    (android_final['Category'] == 'BOOKS_AND_REFERENCE') &
    (android_final['Installs'].isin([1_000_000_000, 500_000_000, 100_000_000]))
]

# Print the app names and their install numbers
for index, row in filtered_apps.iterrows():
    print(f"{row['App']}: {row['Installs']:.0f}")  # Format to display as whole numbers

Amazon Kindle: 100000000
Audiobooks from Audible: 100000000
Bible: 100000000
Google Play Books: 1000000000
Wattpad 📖 Free Books: 100000000


In [37]:
# Filter for BOOKS_AND_REFERENCE apps with fewer than 100 million installs
under_100_m = android_final[(android_final['Category'] == 'BOOKS_AND_REFERENCE') & (android_final['Installs'] < 100000000)]

# Calculate the average installs
average_installs = under_100_m['Installs'].mean()

print(average_installs)

1437212.2162162163


While the average value for books and reference changed from 8,767,811 to 1,437,212, it looks like there are only a few very popular apps, so this market still shows potential due to low competitor amount.

Let's try to get some book app category ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads)

In [38]:
# Convert installs to numeric, if not already done
android_final['Installs'] = pd.to_numeric(android_final['Installs'], errors='coerce')

# Define the install counts you want to check (as numeric values)
install_counts = [1000000.0, 5000000.0, 10000000.0, 50000000.0]

# Filter the DataFrame for the specified category and install counts
filtered_apps = android_final[
    (android_final['Category'] == 'BOOKS_AND_REFERENCE') &
    (android_final['Installs'].isin(install_counts))
]

# Sort the filtered DataFrame by 'Installs' from largest to smallest
filtered_apps = filtered_apps.sort_values(by='Installs', ascending=False)

# Print the app names and install counts
for index, row in filtered_apps.iterrows():
    print(row['App'], ':', row['Installs'])

NOOK: Read eBooks & Magazines : 10000000.0
English Dictionary - Offline : 10000000.0
Dictionary.com: Find Definitions for English Words : 10000000.0
FBReader: Favorite Book Reader : 10000000.0
Dictionary - Merriam-Webster : 10000000.0
Dictionary : 10000000.0
Cool Reader : 10000000.0
HTC Help : 10000000.0
JW Library : 10000000.0
Moon+ Reader : 10000000.0
English Hindi Dictionary : 10000000.0
Oxford Dictionary of English : Free : 10000000.0
Quran for Android : 10000000.0
Aldiko Book Reader : 10000000.0
Spanish English Translator : 10000000.0
Al-Quran (Free) : 10000000.0
Al'Quran Bahasa Indonesia : 10000000.0
Al Quran Indonesia : 10000000.0
Wikipedia : 10000000.0
Read books online : 5000000.0
Al Quran : EAlim - Translations & MP3 Offline : 5000000.0
50000 Free eBooks & Free AudioBooks : 5000000.0
English to Hindi Dictionary : 5000000.0
Ebook Reader : 5000000.0
Bible KJV : 5000000.0
Dictionary - WordWeb : 5000000.0
AlReader -any text book reader : 5000000.0
Ancestry : 5000000.0
eBoox: book

<ipython-input-38-e3777be87483>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  android_final['Installs'] = pd.to_numeric(android_final['Installs'], errors='coerce')


This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book or a more recent book and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

# Conclusions
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book or a more recent book and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.